In [1]:
import os, sys
import numpy as np
# parent_dir = os.path.join(os.getcwd(), os.pardir)
# sys.path.insert(1, os.path.join(parent_dir, "src"))

In [2]:
from elfpy.utils.parse_json import parse_trade, parse_action, parse
import json

In [3]:
from elfpy.simulators import YieldSimulator
from elfpy.pricing_models import ElementPricingModel

random_seed = 3
simulator_rng = np.random.default_rng(random_seed)
config = {
    "min_fee": 0.1, # decimal that assigns fee_percent
    "max_fee": 0.5, # decimal that assigns fee_percent
    "min_target_liquidity": 1e6, # in USD
    "max_target_liquidity": 10e6, # in USD
    "min_target_volume": 0.001, # fraction of pool liquidity
    "max_target_volume": 0.01, # fration of pool liquidity
    "min_pool_apy": 0.02, # as a decimal
    "max_pool_apy": 0.9, # as a decimal
    "pool_apy_target_range": [0.15,0.20], # as a decimal
    "pool_apy_target_range_convergence_speed": 0.52, # as a share of trades that move in convergence direction
    "min_vault_age": 0, # fraction of a year
    "max_vault_age": 1, # fraction of a year
    "min_vault_apy": 0.001, # as a decimal
    "max_vault_apy": 0.9, # as a decimal
    "base_asset_price": 2.5e3, # aka market price
    "pool_duration": 180, # in days
    "num_trading_days": 180, # should be <= pool_duration
    "floor_fee": 0, # minimum fee percentage (bps)
    "tokens": ["base", "fyt"],
    "trade_direction": "out",
    "precision": None,
    "rng": simulator_rng,
    "verbose": False,
    "pricing_model_name": 'HyperDrive',
    "user_policies": "WeightedRandom",
    "num_trading_days": 90,
    "token_duration": 90
}

simulator = YieldSimulator(**config)
simulator.set_random_variables()
simulator.setup_simulated_entities()
market = simulator.market

In [4]:
print(simulator)
print(market)

In [8]:
# assign directory
directory = '../src/elfpy/strategies'

for filename in os.scandir(directory):
    if filename.is_file():
        print(filename.path)
        s = open(filename.path).read()
        print(s)
        l = json.load(open(filename.path))
        print(f"load direct:      {l}")
        l = json.loads(s)
        print(f"load from string: {l}")

../src/elfpy/strategies\simple1.json
{
    "type": "simple_trader",
    "budget": 1000,
    "trade": {
        "buy": {
            "amount": 100,
            "direction": "in"
        }
    }
}
load direct: {'type': 'simple_trader', 'budget': 1000, 'trade': {'buy': {'amount': 100, 'direction': 'in'}}}
load from string: {'type': 'simple_trader', 'budget': 1000, 'trade': {'buy': {'amount': 100, 'direction': 'in'}}}
../src/elfpy/strategies\simple2.json
{
    "type": "simple_trader",
    "budget": 1000,
    "trade": {
        "conditional": {
            "wtrf": {
                "operator": "==",
                "comparator": {
                    "method": "random",
                    "distribution": "integers",
                    "low": 0,
                    "high": 2
                },
                "value": 0
            },
            "then": {"buy": {"amount": 100}},
            "else": {"sell": {"amount": 100}}
        }
    }
}
load direct: {'type': 'simple_trader', 'budget'

JSONDecodeError: Expecting ':' delimiter: line 6 column 18 (char 108)

In [11]:
def parse_dict(d,prefix=''):
    for (k,v) in d.items():
        if isinstance(v, dict):
            print(f"{prefix}{k}:")
            parse_dict(v,f"{prefix} ")
        else:
            print(f"{prefix}{k}: {v} {type(v)}")
parse_dict(l)
print('just the trade key, which is the trade_spec:')
parse_dict(l["trade"])

type: simple_trader <class 'str'>
budget: 1000 <class 'int'>
trade:
 buy:
  amount: 100 <class 'int'>
  direction: in <class 'str'>
just the trade key, which is the trade_spec:
buy:
 amount: 100 <class 'int'>
 direction: in <class 'str'>


In [6]:
policy = l["trade"]
root = list(policy)[0]
policy[root]

{'amount': 100, 'direction': 'in'}

In [7]:
from elfpy.user import User
policy = l
user = User(policy, rng)
t = user.get_trade(market)
print(t)

action_spec: {'buy': {'amount': 100, 'direction': 'in'}}
('base', 'pt', 100)


In [9]:
parse(l["trade"],market,rng)

ValueError: Unrecognized operation buy